# Code used to fine-tune the whole network for each worker in the test set, when k=100

3-fold validation setting

We load the mean aesthetic predictor network, and for each worker, we select 100 images to fine-tune the network with. This is done 3 times and results, in spearman and MSE, are averaged for the 3 validations for each worker. We save the results in a file. We do not save any network at this stage.

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import PIL
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
from random import randint
from sklearn.model_selection import train_test_split


In [ ]:
root_dir = os.path.abspath('')
data_dir = os.path.join(root_dir, 'Images')
# check for existence
os.path.exists(root_dir)
os.path.exists(data_dir)
workers_test = pd.read_csv(os.path.join(data_dir,  'test_workers_dataset_normalized.csv'), sep=' ')

In [ ]:
from PIL import Image
class ImageRatingsDataset(Dataset):
    """Images dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.images_frame = pd.read_csv(csv_file, sep=' ')
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.images_frame)

    def __getitem__(self, idx):
        try:
            img_name = str(os.path.join(self.root_dir,str(self.images_frame.iloc[idx, 0])))
            im = Image.open(img_name).convert('RGB')
            if im.mode == 'P':
                im = im.convert('RGB')
            image = np.asarray(im)
            #image = io.imread(img_name+'.jpg', mode='RGB').convert('RGB')
            rating = self.images_frame.iloc[idx, 1]
            sample = {'image': image, 'rating': rating}

            if self.transform:
                sample = self.transform(sample)
            return sample            
        except Exception as e:
            pass
        

            
        

In [ ]:
import random
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, rating = sample['image'], sample['rating']
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        image = transform.resize(image, (new_h, new_w))

        return {'image': image, 'rating': rating}

        
    

class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, rating = sample['image'], sample['rating']
        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]


        return {'image': image, 'rating': rating}

class RandomHorizontalFlip(object):
    def __init__(self, p):
        self.p = p
    
    def __call__(self, sample):
        
        image, rating = sample['image'], sample['rating']
        if random.random() < self.p:
            image = np.flip(image, 1)
            #image = io.imread(img_name+'.jpg', mode='RGB').convert('RGB')
        return {'image': image, 'rating': rating}

class Normalize(object):
    def __init__(self ):
        self.means=np.array([0.485, 0.456, 0.406]) 
        self.stds=np.array([0.229, 0.224, 0.225])
        
    def __call__(self, sample):
        image, rating = sample['image'], sample['rating']
        im=image/255
        im[:,:,0]=(image[:,:,0] - 0.485)/ 0.229 
        im[:,:,1]=(image[:,:,1]  - self.means[1])/ self.stds[1]
        im[:,:,2]=(image[:,:,2]  - self.means[2])/ self.stds[2]
        image=im
        return {'image': image, 'rating': rating}

    
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, rating = sample['image'], sample['rating']
        
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
            
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).double(),
                'rating': torch.from_numpy(np.float64([rating])).double()}

In [ ]:
worker=workers_test['worker'].unique()[0]
num_images = workers_test[workers_test['worker'].isin([worker])].shape[0]
percent=100/num_images
images=workers_test[workers_test['worker'].isin([worker])][[' imagePair', ' score']]
train_dataframe, valid_dataframe = train_test_split(images, train_size=percent)
train_path="train_means_normalized" + worker +".csv"
test_path="test_means_normalized" + worker +".csv"

In [ ]:
train_dataframe.to_csv(train_path, sep=' ', index_label = False)
valid_dataframe.to_csv(test_path, sep=' ', index_label = False)

In [ ]:
output_size=(224,224)
transformed_dataset_train = ImageRatingsDataset(csv_file=train_path,root_dir='Images/',
                                           transform=transforms.Compose([Rescale(output_size=(256,256)),
                                                                         RandomHorizontalFlip(0.5),
                                                                         RandomCrop(output_size=output_size),
                                                                         Normalize(),
                                                                        ToTensor(),
                                           ]))
transformed_dataset_valid = ImageRatingsDataset(csv_file=test_path,root_dir='Images/',
                                           transform=transforms.Compose([Rescale(output_size=(224,224)),
                                                                         Normalize(),
                                                                        ToTensor(),
                                           ]))

In [ ]:
from torch.utils.data.dataloader import default_collate
bsize=12
def my_collate(batch):
    batch = list(filter (lambda x:x is not None, batch))
    return default_collate(batch)

dataloader_train = DataLoader(transformed_dataset_train, batch_size=bsize,
                        shuffle=True, num_workers=0,collate_fn=my_collate)
dataloader_valid = DataLoader(transformed_dataset_valid, batch_size=8,
                        shuffle=True, num_workers=0,collate_fn=my_collate)


In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import copy
from torch import nn
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
from random import randint

In [ ]:
from torchvision import models
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs,1))

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self, num_classes, keep_probability, inputsize):
    
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(inputsize, 256)
        self.drop_prob = (1 - keep_probability)
        self.relu1= nn.PReLU()
        self.drop1 = nn.Dropout(self.drop_prob)
        self.bn1=nn.BatchNorm1d(256)
        self.fc2=nn.Linear(256,256)
        self.relu2=nn.PReLU()
        self.drop2 = nn.Dropout(p=self.drop_prob)
        self.bn2=nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # Weight initialization reference: https://arxiv.org/abs/1502.01852
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x):
        """
        Feed-forward pass.
        :param x: Input tensor
        : return: Output tensor
        """
        out=self.fc1(x)
        out =self.relu1(out)
        out=self.drop1(out)
        out=self.bn1(out)
        out=self.fc2(out)
        out=self.relu2(out)
        out=self.drop2(out)
        out=self.bn2(out)
        out=self.fc3(out)
        return out

In [ ]:
class convNet(nn.Module):
    #constructor
    def __init__(self,resnet,mynet):
        super(convNet, self).__init__()
        #defining layers in convnet
        self.resnet=resnet
        self.myNet=mynet
    def forward(self, x):
        x=self.resnet(x)
        x=self.myNet(x)
        return x

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.out_features
net1 = BaselineModel(1, 0.5,num_ftrs)
net2=convNet(resnet=model_ft, mynet=net1)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
use_gpu = True
if use_gpu:
    cuda = torch.device('cuda:0')     # Default CUDA device
count=0
torch.cuda.set_device(cuda.index)

In [ ]:
def exp_lr_scheduler(optimizer, epoch, init_lr=0.0001, lr_decay_epoch=2):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""

    if epoch % lr_decay_epoch == 0:
        #print('LR is set to {}'.format(lr))
        pass
    lr = init_lr * (0.9**(epoch // lr_decay_epoch))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [ ]:
net_2 = (torch.load('fine_tuned_flickerAES_normalized_dropout_resnet18_customnetworkadamnormalized.pt'))

In [ ]:
for param in net_2.parameters():
    param.requires_grad=True
net_2.myNet.requires_grad=True

In [ ]:
import time
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
def train_model(model, criterion, optimizer, lr_scheduler,dataloader_train,dataloader_valid,  num_epochs=100):
    since = time.time()
    train_loss_average=[]
    test_loss=[]
    best_model = model
    best_loss = 100
    use_gpu=True
    criterion.cuda()
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['val', 'train']:
            if phase == 'train':
                mode='train'
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
                dataloader=dataloader_train
            else:
                model.eval()
                mode='val'
                dataloader=dataloader_valid

            running_loss = 0.0
            model.cuda()

            counter=0
            # Iterate over data.
            for batch_idx, data in enumerate(dataloader):
                inputs = data['image']
                labels=data['rating']
                if use_gpu:
                    try:
                        inputs, labels = Variable(inputs.float().cuda()), Variable(labels.float().cuda())
                    except:
                        print(inputs,labels)
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                # wrap them in Variable

                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer.zero_grad()
                outputs = model(inputs)
                
                loss = criterion(outputs, labels)
                #print('loss done')                
                # Just so that you can keep track that something's happening and don't feel like the program isn't running.
                #if counter%200==0:
                    #print("Reached iteration ",counter)
                counter+=1

                # backward + optimize only if in training phase
                if phase == 'train':
                    #print('loss backward')
                    loss.backward()
                    #print('done loss backward')
                    optimizer.step()
                    #print('done optim')
                # print evaluation statistics
                try:
                    running_loss += loss.data[0]
                except:
                    print('unexpected error, could not calculate loss or do a sum.')
            #print('trying epoch loss')
            epoch_loss = running_loss / len(dataloader)
            #print('{} Loss: {:.4f} '.format(
            #    phase, epoch_loss))
            if phase == 'train':
                train_loss_average.append(epoch_loss)


            # deep copy the model
            if phase == 'val':
                test_loss.append(epoch_loss)
                if epoch_loss < best_loss:
                    best_loss=epoch_loss
                    best_model = copy.deepcopy(model)
                    print('new best loss = ',epoch_loss)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))
    print('returning and looping back')
    return best_model.cuda(), train_loss_average, test_loss

In [ ]:
from scipy.stats import spearmanr
spearmanr(ratings_i, predictions_i)

In [ ]:
def computeSpearman(dataloader_valid, model):
    ratings = []
    predictions=[]
    #device = cuda
    #criterion = nn.MSELoss()
    #criterion.cuda()
    with torch.no_grad():
        cum_loss=0
        for batch_idx, data in enumerate(dataloader_valid):
            inputs = data['image']
            labels=data['rating']
            if use_gpu:
                try:
                    inputs, labels = Variable(inputs.float().cuda()), Variable(labels.float().cuda())
                except:
                    print(inputs,labels)
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            outputs = model(inputs)
            ratings.append(labels.float())
            predictions.append(outputs.float())
    
    ratings_i=[(list(np.float_([j for j in i]))) for i in ratings]
    predictions_i=[(list(np.float_([j for j in i]))) for i in predictions]
    ratings_i=np.vstack(ratings)
    predictions_i=np.vstack(predictions)
    from scipy.stats import spearmanr
    return spearmanr(ratings_i, predictions_i)

In [ ]:
def ComputeIncreaseSP(sp):
    return max(sp) - sp[0]

def ComputeDecreaseMSE(MSEerrors):
    return min(MSEerrors)-MSEerrors[0]

In [ ]:
import time
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
def train_model(model, criterion, optimizer, lr_scheduler,dataloader_train,dataloader_valid,  num_epochs=100):
    since = time.time()
    train_loss_average=[]
    test_loss=[]
    spearman_test=[]
    best_model = model
    best_loss = 100
    best_spearman=0
    use_gpu=True
    criterion.cuda()
    for epoch in range(num_epochs):
        #print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        #print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['val', 'train']:
            if phase == 'train':
                mode='train'
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
                dataloader=dataloader_train
            else:
                model.eval()
                mode='val'
                dataloader=dataloader_valid

            running_loss = 0.0
            model.cuda()

            counter=0
            # Iterate over data.
            for batch_idx, data in enumerate(dataloader):
                inputs = data['image']
                labels=data['rating']
                if use_gpu:
                    try:
                        inputs, labels = Variable(inputs.float().cuda()), Variable(labels.float().cuda())
                    except:
                        print(inputs,labels)
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                # wrap them in Variable

                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer.zero_grad()
                outputs = model(inputs)
                
                loss = criterion(outputs, labels)
                #print('loss done')                
                # Just so that you can keep track that something's happening and don't feel like the program isn't running.
                #if counter%200==0:
                    #print("Reached iteration ",counter)
                counter+=1

                # backward + optimize only if in training phase
                if phase == 'train':
                    #print('loss backward')
                    loss.backward()
                    #print('done loss backward')
                    optimizer.step()
                    #print('done optim')
                # print evaluation statistics
                try:
                    running_loss += loss.data[0]
                except:
                    print('unexpected error, could not calculate loss or do a sum.')
            #print('trying epoch loss')
            epoch_loss = running_loss / len(dataloader)
            #print('{} Loss: {:.4f} '.format(
            #    phase, epoch_loss))
            if phase == 'train':
                train_loss_average.append(epoch_loss)


            # deep copy the model
            if phase == 'val':
                test_loss.append(epoch_loss)
                sp=computeSpearman(dataloader, model)[0]
                spearman_test.append(sp)

                if epoch_loss < best_loss:
                    best_loss=epoch_loss
                    #print('new best loss = ',epoch_loss)
                
                if sp>best_spearman:
                    best_spearman=sp
                    best_model = copy.deepcopy(model)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))
    print('returning and looping back')
    return best_model.cuda(), train_loss_average, np.asarray(test_loss), np.asarray(spearman_test)

In [ ]:
from torchvision import models
from torch import optim
import torch.nn.functional as F

model_ft=net2


#device = cuda
criterion = nn.MSELoss()
criterion.cuda()
model_ft.cuda()
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)



In [ ]:
ComputeDecreaseMSE(test_loss)

In [ ]:
cols=list(['worker', 'DecreaseMSE', 'IncreaseSP'])
results=pd.DataFrame(columns=cols)

In [ ]:
results=worker, ComputeDecreaseMSE(test_loss), ComputeIncreaseSP(sp)

In [ ]:
results=(np.asarray(results)).tolist()

In [ ]:
results['MSE',0]=[ComputeIncreaseSP(sp)]

In [ ]:
results.append(results)

In [ ]:
results[1]

In [ ]:
resu= []

In [ ]:
resu.append(results)

In [ ]:
resu.append(results)

In [ ]:
resu

In [ ]:
epochs=20
stats=[]
for worker_idx in range(0,37):
    resu= []
    for i in range(0,3):
        worker=workers_test['worker'].unique()[worker_idx]
        print(worker, i)
        num_images = workers_test[workers_test['worker'].isin([worker])].shape[0]
        percent=100/num_images
        images=workers_test[workers_test['worker'].isin([worker])][[' imagePair', ' score']]
        train_dataframe, valid_dataframe = train_test_split(images, train_size=percent)
        train_path="train_means_normalized" + worker +".csv"
        test_path="test_means_normalized" + worker +".csv"
        train_dataframe.to_csv(train_path, sep=' ', index_label = False)
        valid_dataframe.to_csv(test_path, sep=' ', index_label = False)

        output_size=(224,224)
        transformed_dataset_train = ImageRatingsDataset(csv_file=train_path,root_dir='Images/',
                                               transform=transforms.Compose([Rescale(output_size=(256,256)),
                                                                             RandomHorizontalFlip(0.5),
                                                                             RandomCrop(output_size=output_size),
                                                                             Normalize(),
                                                                            ToTensor(),
                                               ]))
        transformed_dataset_valid = ImageRatingsDataset(csv_file=test_path,root_dir='Images/',
                                               transform=transforms.Compose([Rescale(output_size=(224,224)),
                                                                             Normalize(),
                                                                            ToTensor(),
                                               ]))

        from torch.utils.data.dataloader import default_collate
        bsize=30
        def my_collate(batch):
            batch = list(filter (lambda x:x is not None, batch))
            return default_collate(batch)

        dataloader_train = DataLoader(transformed_dataset_train, batch_size=bsize,
                                shuffle=True, num_workers=0,collate_fn=my_collate)
        dataloader_valid = DataLoader(transformed_dataset_valid, batch_size=20,
                                shuffle=True, num_workers=0,collate_fn=my_collate)


        model_ft= (torch.load('fine_tuned_flickerAES_normalized_dropout_resnet18_customnetworkadamnormalized.pt'))



        #device = cuda
        criterion = nn.MSELoss()
        criterion.cuda()
        model_ft.cuda()
        optimizer = optim.Adam(model_ft.parameters(), lr=0.001)

        iteration=0
        network_path= worker + 'network_0 K100full it' + str(i) + '.pt'
        # Run the functions and save the best model in the function model_ft.
        model_ft, train_loss, test_loss, spearman = train_model(model_ft, criterion, optimizer, exp_lr_scheduler,dataloader_train,dataloader_valid,
                               num_epochs=epochs)
 

        #torch.save(model_ft, network_path)
        results=worker, ComputeDecreaseMSE(test_loss), ComputeIncreaseSP(spearman), np.argmin(test_loss), np.argmax(spearman)
        resu.append(results)
        
        
    resupd=pd.DataFrame(resu)
    resupd.columns=['Worker', 'DecreaseMSE', 'IncreaseSpearman', 'ItMSE', 'ItSpearman']
    path=worker+'k100 finetuning all network.csv'
    resupd.to_csv(path, sep=' ', index_label = False)
    number=images[' imagePair'].unique().shape[0]
    stat=resupd['Worker'][0],number, np.mean(resupd['DecreaseMSE']), np.std(resupd['DecreaseMSE']), np.mean(resupd['IncreaseSpearman']), np.std(resupd['IncreaseSpearman']), np.mean(resupd['ItMSE']), np.std(resupd['ItMSE']),np.mean(resupd['ItSpearman']), np.std(resupd['ItSpearman']),
    stats.append(stat)
stats=pd.DataFrame(stats)
stats.to_csv('20 epochs k100 all layers.csv', sep=' ', index_label=False)